## Chatbot Logic

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install -qqq transformers datasets

In [ ]:
import os
import torch
import random
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from IPython.display import Markdown, display
from datasets import Dataset

In [ ]:
try:
    os.chdir("/content/drive/MyDrive/skripsi")
    print("Directory changed")
except OSError:
    print("Error: Can't change the Current Working Directory")

In [ ]:
# Constants
start_message = "==== Hello! I am Alex and I am your virtual friend. If you need a listening ear, I'm always here. To end the chat, input 'exit' in the chatbox. ===="

prevention_messages = ["Are you okay? How long have you been feeling this way?",
                       "That sounds so painful, and I appreciate you sharing that with me. How can I help?",
                       "I know things seem bleak now, but it can be hard to see possible solutions when you feel so overwhelmed.",
                       "I'm concerned about you because I care, and I want to offer support however I can. You can talk to me.",
                       "I'm always here if you feel like talking.",
                       "I'm always here to listen, but do you think a therapist could help a little more?",
                       "Have you thought about talking to a therapist?",
                       "You can withstand any storm and when you are too tired to stand, I will hold you up. You are never alone.",
                       "You know I’m always here for you.",
                       "You’re allowed to have bad days, but remember tomorrow is a brand new day.",
                       "You’ve got a place here on Earth for a reason.",
                       "It's okay to have such thoughts but if they become overwhelming, don't keep it to yourself. I am here for you.",
                       "Everything is a season, and right now you’re in winter. It’s dark and cold and you can’t find shelter, but one day it’ll be summer, and you’ll look back and be grateful you stuck it out through winter.",
                       "I know you are going through a lot and you’re scared, but you will never lose me.",
                       "I know it feels like a lot right now. It’s OK to feel that way.",
                       "Is there anything I can do to make this day go easier for you?"]

helpline_message = "In times of severe distress where you need to speak with someone immediately, these are suicide hotline services available for you. You will be speaking with volunteers or professionals who are trained to deal with suicide crisis. Samaritans of Singapore (SOS; 24 hours): 1800 221 4444 Mental Health Helpline (24 hours): 6389 2222 Singapore Association for Mental Health (SAMH) Helpline: 1800 283 7019"

### Functions

In [ ]:
def printmd(string):
    display(Markdown(string))

In [ ]:
# Untuk load tokenizer dan model generate response
def load_tokenizer_and_model(model="microsoft/DialoGPT-large"):
  tokenizer = AutoTokenizer.from_pretrained(model)
  model = AutoModelForCausalLM.from_pretrained(model)

  return tokenizer, model

In [ ]:
# Untuk load tokenizer dan model sentiment analysis (detect suicidal intention)
def load_suicide_tokenizer_and_model(tokenizer="bert-base-uncased", model='./Models/bert/'):
  suicide_tokenizer = AutoTokenizer.from_pretrained(tokenizer)
  suicide_model = AutoModelForSequenceClassification.from_pretrained('./Models/bert/')

  return suicide_tokenizer, suicide_model

In [ ]:
# Detecting Suicidal Intention
def check_intent(text):
  global suicide_tokenizer, suicide_model
  tokenised_text = suicide_tokenizer.encode_plus(text, return_tensors="pt")
  logits = suicide_model(**tokenised_text)[0]
  prediction = round(torch.softmax(logits, dim=1).tolist()[0][1])
  return prediction

In [ ]:
# Generate Response
# Requires Editing
def generate_response(tokenizer, model, chat_round, chat_history_ids):
  user_input = input(">> You: ")

  if user_input == "exit":
    raise Exception("End of Conversation")

  # Encode user input and End-of-String (EOS) token
  new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

  # Append tokens to chat history
  bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim = -1) if chat_round > 0 else new_input_ids

  # Generate response given maximum chat length history of 1250 tokens
  chat_history_ids = model.generate(bot_input_ids, max_length=1250, pad_token_id=tokenizer.eos_token_id)

  # Print response based on intent
  if check_intent(user_input):
    printmd("*Alex:* {}".format(random.choice(prevention_messages)))
    printmd("{}".format(helpline_message))
  else:
    printmd("*Alex:* {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

  # Return the chat history ids
  return chat_history_ids

In [ ]:
# Initialize chatbot tokenizer and model
tokenizer, model = load_tokenizer_and_model()

# Initialize chatbot history variable
chat_history_ids = None

# Initialise suicide detection tokenizer and model
suicide_tokenizer, suicide_model = load_suicide_tokenizer_and_model()

In [ ]:
# Edit sebagaimana mungkin...
def start_chatbot(n=1000):
  global tokenizer, model, chat_history_ids

  print(start_message)

  try:
    for chat_round in range(n):
      chat_history_ids = generate_response(tokenizer, model, chat_round, chat_history_ids)
  except Exception as e:
    printmd("*Alex:* See ya")


## Serverside (FastAPI)



In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=ed4e78fe1a7b4297c1435cc93d979ef22ca3cf49a2afade071f60d7c3eeca60c
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all 

In [ ]:
messages = ["Are you okay? How long have you been feeling this way?",
                       "That sounds so painful, and I appreciate you sharing that with me. How can I help?",
                       "I know things seem bleak now, but it can be hard to see possible solutions when you feel so overwhelmed.",
                       "I'm concerned about you because I care, and I want to offer support however I can. You can talk to me.",
                       "I'm always here if you feel like talking.",
                       "I'm always here to listen, but do you think a therapist could help a little more?",
                       "Have you thought about talking to a therapist?",
                       "You can withstand any storm and when you are too tired to stand, I will hold you up. You are never alone.",
                       "You know I’m always here for you.",
                       "You’re allowed to have bad days, but remember tomorrow is a brand new day.",
                       "You’ve got a place here on Earth for a reason.",
                       "It's okay to have such thoughts but if they become overwhelming, don't keep it to yourself. I am here for you.",
                       "Everything is a season, and right now you’re in winter. It’s dark and cold and you can’t find shelter, but one day it’ll be summer, and you’ll look back and be grateful you stuck it out through winter.",
                       "I know you are going through a lot and you’re scared, but you will never lose me.",
                       "I know it feels like a lot right now. It’s OK to feel that way.",
                       "Is there anything I can do to make this day go easier for you?"]



In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import random

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/chat/{chat_id}')
async def get_chat(chat_id):
  return {"item_id": chat_id}

@app.get('/chat/send/{send}')
async def send_chat(send):
  return {"response": random.choice(messages)}

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [534]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://9a07-34-74-209-146.ngrok.io
INFO:     103.195.142.239:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     103.195.142.239:0 - "GET /chat/send/hello HTTP/1.1" 200 OK
INFO:     103.195.142.239:0 - "GET /chat/send/yeah HTTP/1.1" 200 OK
INFO:     103.195.142.239:0 - "GET /chat/send/test HTTP/1.1" 200 OK
INFO:     103.195.142.239:0 - "GET /chat/send/hello HTTP/1.1" 200 OK
INFO:     103.195.142.239:0 - "GET /chat/send/something HTTP/1.1" 200 OK
INFO:     103.195.142.239:0 - "GET /chat/send/test HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [534]


In [ ]:
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
import uvicorn
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

templates = Jinja2Templates(directory="templates")

english_bot = ChatBot("Chatterbot")
trainer = ChatterBotCorpusTrainer(english_bot)
trainer.train("chatterbot.corpus.english")

@app.get("/", response_class=HTMLResponse)
def home(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

@app.get("/getChatBotResponse")
def get_bot_response(msg: str):
    return str(english_bot.get_response(msg))

ModuleNotFoundError: ignored